In [69]:
# import necessary libraries
import pandas as pd
import questionary
from pathlib import Path
import fire 
import numpy as np
%matplotlib inline

In [335]:
# open csv files using read_csv function
AUDUSD_df = pd.read_csv(
            Path('../Resources/AUDUSD-2000-2020-15m.csv'),
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)
EURCHF_df = pd.read_csv(
            Path('../Resources/EURCHF-2000-2020-15m.csv'), 
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)
EURJPY_df = pd.read_csv(
            Path('../Resources/EURJPY-2000-2020-15m.csv'), 
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)
EURUSD_df = pd.read_csv(
            Path('../Resources/EURUSD-2000-2020-15m.csv'), 
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)
USDCAD_df = pd.read_csv(
            Path('../Resources/USDCAD-2000-2020-15m.csv'), 
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)
USDCHF_df = pd.read_csv(
            Path('../Resources/USDCHF-2000-2020-15m.csv'), 
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)
USDJPY_df = pd.read_csv(
            Path('../Resources/USDJPY-2000-2020-15m.csv'), 
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)


In [336]:
USDCAD_df.drop(['HIGH', 'LOW', 'OPEN'], axis=1, inplace=True)

In [337]:
#Toggles for the SMA windows. Remember data is in 15 minutes intervals and 24 hours a day. 1 day = 96
fast_sma = 96
slow_sma = 672
USDCAD_df['fast sma'] = USDCAD_df['CLOSE'].rolling(window=fast_sma).mean()
USDCAD_df['slow sma'] = USDCAD_df['CLOSE'].rolling(window=slow_sma).mean()

#adds percent change and cumulative return column for the currency
USDCAD_df['pct_change'] = USDCAD_df['CLOSE'].pct_change()
USDCAD_df.dropna(inplace=True)
USDCAD_df['cum returns'] = (1 + USDCAD_df['pct_change']).cumprod()
USDCAD_df

,CLOSE,fast sma,slow sma,pct_change,cum returns
DATE_TIME,,,,,
2000-01-12 04:15:00,1.45680,1.456460,1.453219,0.000206,1.000206
2000-01-12 04:30:00,1.45670,1.456473,1.453235,-0.000069,1.000137
2000-01-12 05:00:00,1.45660,1.456485,1.453250,-0.000069,1.000069
2000-01-12 05:15:00,1.45660,1.456498,1.453266,0.000000,1.000069
2000-01-12 05:30:00,1.45670,1.456511,1.453282,0.000069,1.000137
...,...,...,...,...,...
2020-03-19 06:15:00,1.46175,1.443844,1.395372,-0.001073,1.003605
2020-03-19 06:30:00,1.46145,1.444284,1.395517,-0.000205,1.003399
2020-03-19 06:45:00,1.46216,1.444740,1.395663,0.000486,1.003886


In [338]:
#Adds long/short column. A 1 indicates the strategy is long and a 0 is short.
USDCAD_df['long/short'] = np.where(USDCAD_df['fast sma'] > USDCAD_df['slow sma'],1,0)
USDCAD_df['strat return'] = np.where(USDCAD_df['long/short'] == 1,USDCAD_df['pct_change'],USDCAD_df['pct_change'] * -1)
USDCAD_df.dropna(inplace=True)

In [339]:
#calculates cumulative returns for the strategy
USDCAD_df['cum strat returns'] = (1 + USDCAD_df['strat return']).cumprod()
USDCAD_df

,CLOSE,fast sma,slow sma,pct_change,cum returns,long/short,strat return,cum strat returns
DATE_TIME,,,,,,,,
2000-01-12 04:15:00,1.45680,1.456460,1.453219,0.000206,1.000206,1,0.000206,1.000206
2000-01-12 04:30:00,1.45670,1.456473,1.453235,-0.000069,1.000137,1,-0.000069,1.000137
2000-01-12 05:00:00,1.45660,1.456485,1.453250,-0.000069,1.000069,1,-0.000069,1.000069
2000-01-12 05:15:00,1.45660,1.456498,1.453266,0.000000,1.000069,1,0.000000,1.000069
2000-01-12 05:30:00,1.45670,1.456511,1.453282,0.000069,1.000137,1,0.000069,1.000137
...,...,...,...,...,...,...,...,...
2020-03-19 06:15:00,1.46175,1.443844,1.395372,-0.001073,1.003605,1,-0.001073,1.180973
2020-03-19 06:30:00,1.46145,1.444284,1.395517,-0.000205,1.003399,1,-0.000205,1.180731
2020-03-19 06:45:00,1.46216,1.444740,1.395663,0.000486,1.003886,1,0.000486,1.181304
